# Testing per image strip processing

This notebook is aimed at prototyping the framework for slicing/windowing a large image into small windows, running inference, and then re-constructing it efficiently.

In [1]:
import os.path as op
import gdal
import rasterio
from skimage.transform import rescale
import tensorflow as tf
import io
import PIL

from divdet.inference.utils_inference import (iter_grouper, get_slice_bounds, windowed_reads_numpy)
from divdet.inference.inference_runner import pred_generator

image_fpath_orig = '/Users/wronk/Data/divot_detect/B04_011293_1265_XN_53S071W.tiff'
output_fpath = '/Users/wronk/Data/divot_detect/B04_011293_1265_XN_53S071W_warp.tiff'

# One message per image
message = dict(scales=[0.25], 
               window_size=(1024, 1024),
               min_window_overlap=(256, 256),
               endpoint='http://localhost:8501/v1/models/divdet-inference:predict',
               batch_size=1)

## Reprojection

In [2]:
image_fpath = op.join(op.splitext(image_fpath_orig)[0] + '_warp' + 
                      op.splitext(image_fpath_orig)[1])
print(f'Reprojecting image and saving to {image_fpath}.')

with rasterio.open(image_fpath_orig) as temp_img:
    center_lat = temp_img.lnglat()[1]  # Raster center
eqc_proj = f'+proj=eqc +lat_ts={center_lat} +lat_0=0 +lon_0=180 +x_0=0 +y_0=0 +a=3396190 +b=3396190 +units=m +no_defs'
gdal.Warp(image_fpath, image_fpath_orig, dstSRS=eqc_proj)
print('Reprojection complete.')

Reprojecting image and saving to /Users/wronk/Data/divot_detect/B04_011293_1265_XN_53S071W_warp.tiff.
Reprojection complete.


# Load local model

In [3]:
saved_model_path = '/Users/wronk/Builds/divdet/models/10/export/saved_model/1'
#detect_fn = tf.saved_model.load(saved_model_path)

## Slice image and run prediction

In [ ]:
preds = []
with rasterio.open(image_fpath) as dataset:
    image = dataset.read(1)

    for scale in message['scales']:
        print(f'Processing at downscale {scale}')

        # Rescale image, round, and convert back to orig datatype
        scaled_image = rescale(image, (scale, scale), anti_aliasing=True).round().astype(image.dtype)

        # Calculate slice bounds and create generator
        slice_bounds = get_slice_bounds(
            scaled_image.shape, slice_size=message.get('window_size'),
            min_window_overlap=message.get('min_window_overlap'))

        slice_gen = windowed_reads_numpy(scaled_image, slice_bounds)
        
        # Run in memory model
        if True:
            byte_io = io.BytesIO()
            image = PIL.Image.fromarray(slice_gen[0]['image_data'])
            image.save(byte_io, 'PNG')
            str_image = [byte_io.getvalue()]
            input_tensor = tf.constant(str_image)
    
        import ipdb; ipdb.set_trace()
        # Run prediction on in-mem model
        #detections = detect_fn(input_tensor)
        
        # Calculate slice bounds and create generator
        pred_gen = pred_generator(slice_gen, message.get('endpoint'), 
                                  message.get('batch_size'))

        preds.extend(list(pred_gen))

Processing at downscale 0.25


100%|██████████| 12/12 [00:00<00:00, 17044.24it/s]


> <ipython-input-4-2b971c246c4a>(31)<module>()
     30         # Calculate slice bounds and create generator
---> 31         pred_gen = pred_generator(slice_gen, message.get('endpoint'), 
     32                                   message.get('batch_size'))

ipdb> n
> <ipython-input-4-2b971c246c4a>(32)<module>()
     31         pred_gen = pred_generator(slice_gen, message.get('endpoint'), 
---> 32                                   message.get('batch_size'))
     33 

ipdb> n
> <ipython-input-4-2b971c246c4a>(34)<module>()
     32                                   message.get('batch_size'))
     33 
---> 34         preds.extend(list(pred_gen))

ipdb> n


0it [00:00, ?it/s]

> /Users/wronk/Builds/divdet/divdet/inference/inference_runner.py(162)pred_generator()
    161         import ipdb; ipdb.set_trace()
--> 162         resp = requests.post(endpoint, data=payload)
    163         #with open('data.txt', 'w') as outfile:

ipdb> n
> /Users/wronk/Builds/divdet/divdet/inference/inference_runner.py(165)pred_generator()
    164         #    json.dump(resp.json(), outfile)
--> 165         preds = json.loads(resp.content)['outputs']['detection_scores']
    166 

ipdb> pred_info = json.loads(resp.content)
ipdb> type(pred_info)
<class 'dict'>
ipdb> pred_info.keys()
dict_keys(['outputs'])
ipdb> len(pred_info)['outputs']
*** TypeError: 'int' object is not subscriptable
ipdb> len(pred_info['outputs'])
23
ipdb> type(pred_info['outputs'])
<class 'dict'>
ipdb> pred_info['outputs'].keys()
dict_keys(['detection_masks', 'rpn_features_to_crop', 'detection_anchor_indices', 'refined_box_encodings', 'final_anchors', 'mask_predictions', 'detection_classes', 'num_detections', 'rpn

ipdb> pred_info.keys()
dict_keys(['outputs'])
ipdb> pred_info = pred_info['outputs']
ipdb> pred_info.keys()
dict_keys(['detection_masks', 'rpn_features_to_crop', 'detection_anchor_indices', 'refined_box_encodings', 'final_anchors', 'mask_predictions', 'detection_classes', 'num_detections', 'rpn_box_predictor_features', 'class_predictions_with_background', 'proposal_boxes', 'raw_detection_boxes', 'rpn_box_encodings', 'box_classifier_features', 'raw_detection_scores', 'proposal_boxes_normalized', 'detection_multiclass_scores', 'anchors', 'num_proposals', 'detection_boxes', 'image_shape', 'rpn_objectness_predictions_with_background', 'detection_scores'])
ipdb> pred_info['mask_predictions'].shape
*** AttributeError: 'list' object has no attribute 'shape'
ipdb> pred_info['mask_predictions'][0].shape
*** AttributeError: 'list' object has no attribute 'shape'
ipdb> pred_info['mask_predictions'][0]
[[[-1.35307074, -1.7539866, -1.9113766, -1.92674053, -1.88205504, -1.81799495, -1.74544883, -1.6

ipdb> temp = np.array(pred_info['mask_predictions'][0])
ipdb> temp.shape
(2, 33, 33)
ipdb> temp = np.array(pred_info['detection_masks'][0][0])
ipdb> temp
array([[0.20536882, 0.14754507, 0.12882629, ..., 0.12637922, 0.15210238,
        0.21923792],
       [0.15411425, 0.08932608, 0.07267588, ..., 0.07018515, 0.09304982,
        0.16844851],
       [0.13963613, 0.07603341, 0.06247002, ..., 0.05935642, 0.07850435,
        0.15228116],
       ...,
       [0.16751674, 0.10684481, 0.0971846 , ..., 0.11825848, 0.11151129,
        0.1723797 ],
       [0.19206882, 0.13414833, 0.12372404, ..., 0.13891536, 0.13657183,
        0.19517034],
       [0.25124478, 0.20966193, 0.20156231, ..., 0.21112794, 0.20926362,
        0.24960551]])
ipdb> temp.shape
(33, 33)
ipdb> pred_info.keys()
dict_keys(['detection_masks', 'rpn_features_to_crop', 'detection_anchor_indices', 'refined_box_encodings', 'final_anchors', 'mask_predictions', 'detection_classes', 'num_detections', 'rpn_box_predictor_features', 'class_

ipdb> len(pred_info['detection_scores'][0])
100
ipdb> temp = pred_info['detection_masks'][0]
ipdb> temp.shape
*** AttributeError: 'list' object has no attribute 'shape'
ipdb> np.array(temp).shape
(100, 33, 33)
ipdb> pred_info.keys
<built-in method keys of dict object at 0x133572990>
ipdb> pred_info.keys()
dict_keys(['detection_masks', 'rpn_features_to_crop', 'detection_anchor_indices', 'refined_box_encodings', 'final_anchors', 'mask_predictions', 'detection_classes', 'num_detections', 'rpn_box_predictor_features', 'class_predictions_with_background', 'proposal_boxes', 'raw_detection_boxes', 'rpn_box_encodings', 'box_classifier_features', 'raw_detection_scores', 'proposal_boxes_normalized', 'detection_multiclass_scores', 'anchors', 'num_proposals', 'detection_boxes', 'image_shape', 'rpn_objectness_predictions_with_background', 'detection_scores'])
ipdb> np.array(pred_info['mask_predictions'][0]).shape
(2, 33, 33)
ipdb> pred_info['num_detections']
[0.0]
ipdb> type(pred_info['refined_box_